In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd '/content/drive/My Drive/data1'

/content/drive/My Drive/data1


In [0]:
import os
data = []
labels = []
classes = 3
cur_path = os.getcwd()

In [0]:
!pip install pyyaml h5py

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D,MaxPool2D,Flatten
from sklearn.model_selection import train_test_split

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image   

In [0]:
traindir='/content/drive/My Drive/data1/train'
valdir='/content/drive/My Drive/data1/val'
image = np.array(traindir)
data.append(image)
labels.append(classes)


In [0]:
#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

(1,) (1,)


In [0]:
classifier = Sequential()

# convolution layer - 32 feature detectors of 3x3 shape plus a rectifier on top of our convolved feature map
classifier.add(Conv2D(32, (3,3), input_shape=(64, 64, 3), activation='relu'))

# max pooling layer - of 2x2 shape
classifier.add(MaxPool2D(pool_size=(2, 2)))

# another convolution layer
classifier.add(Conv2D(32, (3,3), activation='relu'))

# another max pooling layer
classifier.add(MaxPool2D(pool_size=(2, 2)))

# flattening layer
classifier.add(Flatten())

# fully connected layer
classifier.add(Dense(units=128, activation='relu'))

# output layer
classifier.add(Dense(units=3, activation='softmax'))

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)

val_datagen = ImageDataGenerator( rescale=1./255)

training_set = train_datagen.flow_from_directory(directory=traindir,
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='categorical'
                                                )
val_set = val_datagen.flow_from_directory(directory=valdir,
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode='categorical')

Found 330 images belonging to 3 classes.
Found 4 images belonging to 3 classes.


In [0]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
import os
from keras.callbacks import ModelCheckpoint
filepath = "/content/drive/My Drive/data/saved/weights-improvement-{epoch:02d}-{loss: .4f}.hdf5"
checkpoint_dir = os.path.dirname(filepath)

# Create a callback that saves the model's weights
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
history=classifier.fit_generator(training_set,
                         steps_per_epoch=10,                         
                         epochs=5,                                 
                         validation_data=val_set,
                         validation_steps=2500,                         
                         callbacks=callbacks_list)   

Epoch 1/5
10/10 [==============================] - 252s 25s/step - loss: 0.0950 - accuracy: 0.9966 - val_loss: 15.3405 - val_accuracy: 0.2500

Epoch 00001: loss improved from inf to 0.09283, saving model to /content/drive/My Drive/data/saved/weights-improvement-01- 0.0928.hdf5
Epoch 2/5
10/10 [==============================] - 252s 25s/step - loss: 0.0827 - accuracy: 0.9875 - val_loss: 15.1020 - val_accuracy: 0.2500

Epoch 00002: loss improved from 0.09283 to 0.08266, saving model to /content/drive/My Drive/data/saved/weights-improvement-02- 0.0827.hdf5
Epoch 3/5
10/10 [==============================] - 252s 25s/step - loss: 0.0490 - accuracy: 0.9964 - val_loss: 14.9094 - val_accuracy: 0.2500

Epoch 00003: loss improved from 0.08266 to 0.05030, saving model to /content/drive/My Drive/data/saved/weights-improvement-03- 0.0503.hdf5
Epoch 4/5
10/10 [==============================] - 252s 25s/step - loss: 0.0241 - accuracy: 1.0000 - val_loss: 16.4444 - val_accuracy: 0.2500

Epoch 00004: lo

In [0]:
training_set.class_indices
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                

In [0]:

new=classifier.predict(val_set)

In [0]:

test_steps_per_epoch = np.math.ceil(val_set.samples / val_set.batch_size)

predictions = classifier.predict_generator(val_set, steps=test_steps_per_epoch)
print(predictions)



[[9.9049117e-03 9.9009502e-01 4.2966897e-13]
 [9.9999094e-01 9.0126450e-06 2.0999268e-10]
 [1.0000000e+00 1.5275727e-10 4.6295728e-12]
 [9.9999940e-01 6.1577919e-07 1.2761465e-10]]


In [0]:
# Evaluate the restored model
loss, acc = classifier.evaluate(training_set, verbose=2)
print(loss,acc)
loss1, acc1 = classifier.evaluate(val_set, verbose=2)
print(loss1,acc1)


0.0011405833065509796 1.0
16.344579696655273 0.25


In [0]:

def test_unseen(filename):
    test_image =  image.load_img(filename, target_size = (64, 64))
    # converting to a 3D array
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = classifier.predict(test_image)
    
    if result[0][0] == 1:
        prediction = "10"
    elif result[0][0] == 2:
        prediction = "20"
    else:
        prediction = "30"
        
    return prediction

In [0]:
import os

import glob

path = '/content/drive/My Drive/data/val/'
for infile in glob.glob( os.path.join(path, '*.jpg') ):
   print(test_unseen(infile))


In [0]:
!mkdir -p saved_model
classifier.save('/saved')